<a href="https://colab.research.google.com/github/josh-english/DataVisualization/blob/master/Midterm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1 - Data Preprocessing

## Get the Data

In [27]:
# get data from gensim
import gensim.downloader as api

print(api.load("20-newsgroups", return_path=False))
PATH = "/root/gensim-data/20-newsgroups/20-newsgroups.gz"

# unzip data to pandas dataframe
import pandas as pd
df = pd.read_json(PATH, lines=True, compression="gzip")

print(df.shape)
print(df[0:5])

<20-newsgroups.Dataset object at 0x7f16bc8b5a20>
(18846, 4)
                                                data     id    set  \
0  From: db7n+@andrew.cmu.edu (D. Andrew Byler)\n...  21408  train   
1  From: mpaul@unl.edu (marxhausen paul)\nSubject...  21388  train   
2  From: eggertj@moses.atc.ll.mit.edu (Jim Eggert...  20600  train   
3  From: tas@pegasus.com (Len Howard)\nSubject: R...  20820  train   
4  From: db7n+@andrew.cmu.edu (D. Andrew Byler)\n...  21335  train   

                    topic  
0  soc.religion.christian  
1  soc.religion.christian  
2  soc.religion.christian  
3  soc.religion.christian  
4  soc.religion.christian  
